In [1]:
def parse_judge(judgement):   
    if judgement!="not given":
            judgement=judgement.split("\n")[0].split(":")[1].strip().lower()    
    if judgement=="corretto":
        judgement="correct"
    if judgement in "parzialmente corretto":
        judgement="partially correct"
    if judgement in ["incorrect","incorretto","errato","errato","incorreto","incorrente","inccorrect","incorrect","incorretto","incoretto","errata"]:
        judgement="incorrect"
    return judgement

In [2]:
def create_df(path):
    eval_df=pd.read_json(path)
    eval_df["judgement"]=eval_df["judgement"].apply(lambda x :parse_judge(x))
    return eval_df

In [4]:
def compute_f1_strict(df):
    correct=len(df[df["judgement"]=="correct"])
    incorrect=len(df[df["judgement"]=="incorrect"])
    partial=len(df[df["judgement"]=="partially correct"])
    not_given=len(df[df["judgement"]=="not given"])    
    TP=correct
    FP= incorrect + partial 
    FN=not_given
    if TP>0:            
        precision=TP/(TP+FP)
        recall=TP/(TP+FN)
        f1=(2*(precision*recall))/(precision+recall)
    else:
        f1=0.0
    
    return round(f1,2) , [(round(correct/df.shape[0],2)),(round(incorrect/df.shape[0],2)),(round(partial/df.shape[0],2)),(round(not_given/df.shape[0],2))]

In [6]:
def compute_f1_soft(df):
    correct=len(df[df["judgement"]=="correct"])
    incorrect=len(df[df["judgement"]=="incorrect"])
    partial=len(df[df["judgement"]=="partially correct"])
    not_given=len(df[df["judgement"]=="not given"])    
    
    TP=correct + partial
    FP=incorrect 
    FN=not_given
    
    if TP>0:            
        precision=TP/(TP+FP)
        recall=TP/(TP+FN)
        f1=(2*(precision*recall))/(precision+recall)
    else:
        f1=0.0
    
    return round(f1,2) , [(round(correct/df.shape[0],2)),(round(incorrect/df.shape[0],2)),(round(partial/df.shape[0],2)),(round(not_given/df.shape[0],2))]

In [7]:
def compute_acc(df):
    correct=len(df[df["judgement"]=="correct"])
    incorrect=len(df[df["judgement"]=="incorrect"])
    partial=len(df[df["judgement"]=="partially correct"])
    not_given=len(df[df["judgement"]=="not given"])    
    
    TP=correct + partial
    FP=incorrect 
    FN=not_given

    acc_soft=TP/(TP+FP+FN)
    TP=TP-partial
    FP=FP+partial
    acc_strict=TP/(TP+FP+FN)

    return round(acc_strict,2), round(acc_soft,2)
    
    

In [8]:
import numpy as np
import json
import os
from glob import glob
import pandas as pd
model_names=["llama3","gpt","EuroLLM","Minerva"]
results=[]
runs=3
langs=["it","en"]
for model_name in model_names:
    print(f"###### EVALUATING {model_name} ###########")    
    model_df=pd.DataFrame(columns=["dataset","modality","precision","recall","f1","accuracy","correct(%)","incorrect(%)","partial(%)","not given(%)"])
    for lan in langs:
        f1_strict_vals,f1_soft_vals=[],[]
        acc_strict_vals,acc_soft_vals=[],[]
        mean_percentages=[]
        print("LANG",lan)
        for i in range(runs):               
            predictions=[x for x in glob(f"./predictions/{model_name}/run_{i}/*.json") if lan in x]
         
            for pred in predictions:            
                name=os.path.basename(pred).replace(".json","")
                
                df=create_df(pred)           
                f1_strict,percentages=compute_f1_strict(df)
                
                f1_soft,_=compute_f1_soft(df)

                acc_strict,acc_soft=compute_acc(df)               
                acc_strict_vals.append(acc_strict)
                acc_soft_vals.append(acc_soft)
                
                
                f1_strict_vals.append(f1_strict)
                f1_soft_vals.append(f1_soft)
                mean_percentages.append(percentages)
        
        f1_strict_mean=round(np.mean(f1_strict_vals),2)
        f1_soft_mean=round(np.mean(f1_soft_vals),2)

        acc_strict_mean=round(np.mean(acc_strict_vals),2)
        acc_soft_mean=round(np.mean(acc_soft_vals),2)
        
        percent_mean=[round(x,2) for x in np.mean(np.array(mean_percentages),axis=0)]
        
        
        print(f"Strict--F1:{f1_strict_mean}({round(np.std(f1_strict_vals),2)})-Acc:{acc_strict_mean}({round(np.std(acc_strict_vals),2)})")
        print(f"Soft--F1:{f1_soft_mean}({round(np.std(f1_soft_vals),2)})-Acc:{acc_soft_mean}({round(np.std(acc_soft_vals),2)})")
        correct,incorrect,partial,not_given=percent_mean
        print(f"Correct:{correct}--Incorrect:{incorrect}--Partial:{partial}--Not given:{not_given}")    
               
        
       
            

###### EVALUATING llama3 ###########
LANG it
Strict--F1:0.48(0.0)-Acc:0.31(0.0)
Soft--F1:0.69(0.0)-Acc:0.52(0.01)
Correct:0.31--Incorrect:0.42--Partial:0.21--Not given:0.06
LANG en
Strict--F1:0.49(0.01)-Acc:0.32(0.0)
Soft--F1:0.7(0.0)-Acc:0.54(0.0)
Correct:0.32--Incorrect:0.32--Partial:0.22--Not given:0.14
###### EVALUATING gpt ###########
LANG it
Strict--F1:0.33(0.0)-Acc:0.2(0.0)
Soft--F1:0.5(0.0)-Acc:0.33(0.0)
Correct:0.2--Incorrect:0.12--Partial:0.13--Not given:0.55
LANG en
Strict--F1:0.34(0.04)-Acc:0.21(0.03)
Soft--F1:0.53(0.03)-Acc:0.36(0.03)
Correct:0.21--Incorrect:0.28--Partial:0.15--Not given:0.36
###### EVALUATING EuroLLM ###########
LANG it
Strict--F1:0.41(0.0)-Acc:0.26(0.0)
Soft--F1:0.64(0.0)-Acc:0.47(0.0)
Correct:0.26--Incorrect:0.53--Partial:0.21--Not given:0.0
LANG en
Strict--F1:0.29(0.0)-Acc:0.17(0.0)
Soft--F1:0.49(0.0)-Acc:0.32(0.0)
Correct:0.17--Incorrect:0.34--Partial:0.16--Not given:0.34
###### EVALUATING Minerva ###########
LANG it
Strict--F1:0.15(0.0)-Acc:0.08(0.0)